get resnet50 V2 freeze all convolution layer 

In [50]:
import tensorflow as tf
from keras.applications import resnet_v2
from keras.models import Model
import keras

resnet = resnet_v2.ResNet50V2(include_top=False)

#output = resnet.layers[-1].output
#output = keras.layers.Flatten()(output)
for layer in resnet.layers:

    layer.trainable = False
    
import pandas as pd
pd.set_option('max_colwidth', -1)
layers = [(layer, layer.name, layer.trainable) for layer in resnet.layers]
pd.DataFrame(layers, columns=['Layer Type', 'Layer Name', 'Layer Trainable'])    

print(resnet.output_shape)

(None, None, None, 2048)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  from ipykernel import kernelapp as app


In [8]:
from google.colab import drive
drive.mount('/content/drive')
!unzip  "/content/drive/My Drive/Colab Notebooks/mhist_dataset.zip" 

Mounted at /content/drive
Archive:  /content/drive/My Drive/Colab Notebooks/mhist_dataset.zip
   creating: mhist_dataset/
  inflating: mhist_dataset/annotations.csv  
   creating: mhist_dataset/images/
  inflating: mhist_dataset/images/MHIST_aaa.png  
  inflating: mhist_dataset/images/MHIST_aab.png  
  inflating: mhist_dataset/images/MHIST_aac.png  
  inflating: mhist_dataset/images/MHIST_aad.png  
  inflating: mhist_dataset/images/MHIST_aae.png  
  inflating: mhist_dataset/images/MHIST_aaf.png  
  inflating: mhist_dataset/images/MHIST_aag.png  
  inflating: mhist_dataset/images/MHIST_aah.png  
  inflating: mhist_dataset/images/MHIST_aai.png  
  inflating: mhist_dataset/images/MHIST_aaj.png  
  inflating: mhist_dataset/images/MHIST_aak.png  
  inflating: mhist_dataset/images/MHIST_aal.png  
  inflating: mhist_dataset/images/MHIST_aam.png  
  inflating: mhist_dataset/images/MHIST_aan.png  
  inflating: mhist_dataset/images/MHIST_aao.png  
  inflating: mhist_dataset/images/MHIST_aap.png 

Read in data from annotation and png 

In [67]:
import csv
# csv file name
filename = "mhist_dataset/annotations.csv"
import cv2
 
# initializing the titles and rows list
fields = []
rows = []
path = "mhist_dataset/images/"
 
# reading csv file
with open(filename, 'r') as csvfile:
  # creating a csv reader object
  csvreader = csv.reader(csvfile)
     
  # extracting field names through first row
  fields = next(csvreader)
 
  # extracting each data row one by one
  for row in csvreader:
    rows.append(row)
 


#label HP = 0 SSA = 1


training_image = []
training_label = []
testing_image = []
testing_label = []

for row in rows[:100]:
  # parsing each column of a row
  img_path = path + row[0]
  image = cv2.imread(img_path)
  label = [1, 0]
  if row[1] == "SSA":
    label = [0, 1]
    
  if row[3] == "train":
    training_image.append(image)
    training_label.append(label)

  elif row[3] == "test":
    testing_image.append(image)
    testing_label.append(label)

  
print(len(training_label))
print(len(testing_label))






77
23


In [37]:
import numpy as np
print(np.array(training_image).shape)

(2175, 224, 224, 3)


In [68]:
def get_features_from_Resnet(model, input):
  features = model.predict(input, verbose=0)
  return features

training_image = tf.convert_to_tensor(np.array(training_image))
testing_image = tf.convert_to_tensor(np.array(testing_image))

train_features = get_features_from_Resnet(resnet, training_image)
test_features = get_features_from_Resnet(resnet, testing_image)

In [70]:
from keras.layers import Dense, GlobalAveragePooling2D, InputLayer
from keras.models import Sequential
from keras import optimizers

print(train_features.shape[1:])
input_shape = train_features.shape
model = Sequential()
model.add(InputLayer(input_shape=(input_shape[1:])))
model.add(GlobalAveragePooling2D())

# output is binary 
model.add(Dense(2))

model.compile(loss=keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=optimizers.Adam(learning_rate=1e-4),
              metrics=[tf.keras.metrics.AUC(from_logits=True)])
model.summary()


training_label = tf.convert_to_tensor(np.array(training_label))
testing_label = tf.convert_to_tensor(np.array(testing_label))

print(training_label.shape)


history = model.fit(x=train_features, y=training_label,
                    validation_data=(test_features, testing_label),
                    batch_size=32,
                    epochs=10,
                    verbose=1)



(7, 7, 2048)
Model: "sequential_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 global_average_pooling2d_13  (None, 2048)             0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dense_12 (Dense)            (None, 2)                 4098      
                                                                 
Total params: 4,098
Trainable params: 4,098
Non-trainable params: 0
_________________________________________________________________
(77, 2)
Epoch 1/10
3/3 [==============================] - 1s 137ms/step - loss: 115.7104 - auc: 0.4018 - val_loss: 120.8685 - val_auc: 0.3043
Epoch 2/10
3/3 [==============================] - 0s 24ms/step - loss: 107.5448 - auc: 0.4248 - val_loss: 111.7115 - val_auc: 0.3696
Epoch 3/10
3/3 [==============================] - 0s 23ms/step - loss: 99.9697 - auc: 0.4198 